# 3. Working With Pinecone Vector Database
Although Vector Databases have existed long before Large Language Models, vector DBs have become an important part of many LLM solutions. In particular, Retreival Augmented Generation (or RAG) architecture addresses LLM's halucinations and issues with longer-term memory by augmenting the user's prompt with the results of a search accross a vector DB. [Pinecone](https://www.pinecone.io/) is a cloud-based vector database that is easy to integrate with your CML workflow, as this notebook shows. 

Recall that in the previous exervice you created CML jobs to scrape a site and load each page into Pinecone vector DB. This notebook will focus on interacting with Pinecone from a Jupyter notebook.

![Exercise 3 overview](../assets/exercise_3.png)

### 3.1 Imports and global vars

In [1]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')
dimension = 768

### 3.2 Initialize Pinecone connection
Pinecone client is initialized with the parameters defined previously. 

In [2]:
print("initialising Pinecone connection...")
pinecone = Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone initialised")

print(f"Getting '{PINECONE_INDEX}' as object...")
index = pinecone.Index(PINECONE_INDEX)
print("Success")

# Get latest statistics from index
current_collection_stats = index.describe_index_stats()
print('Total number of embeddings in Pinecone index is {}.'.format(current_collection_stats.get('total_vector_count')))

initialising Pinecone connection...
Pinecone initialised
Getting 'llm-hol' as object...
Success
Total number of embeddings in Pinecone index is 15.


### 3.3 Function to peform the vector search 
The idea is to find a chunk from the Knowledge Base that is "close" to what the original user's prompt is. We perform a semantic search using the user's question, find the nearest knowledge base chunk, and return the content of that chunk along with its source and score.

In [3]:
# Get embeddings for a user question and query Pinecone vector DB for nearest knowledge base chunk
def get_nearest_chunk_from_pinecone_vectordb(index, question):
    # Generate embedding for user question with embedding model
    retriever = SentenceTransformer(EMBEDDING_MODEL_REPO)
    xq = retriever.encode([question]).tolist()
    xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)
    
    matching_files = []
    scores = []
    for match in xc['matches']:
        # extract the 'file_path' within 'metadata'
        file_path = match['metadata']['file_path']
        # extract the individual scores for each vector
        score = match['score']
        scores.append(score)
        matching_files.append(file_path)

    # Return text of the nearest knowledge base chunk 
    # Note that this ONLY uses the first matching document for semantic search. matching_files holds the top results so you can increase this if desired.
    response = load_context_chunk_from_data(matching_files[0])
    sources = matching_files[0]
    score = scores[0]
    return response, sources, score

# Return the Knowledge Base doc based on Knowledge Base ID (relative file path)
def load_context_chunk_from_data(id_path):
    with open(id_path, "r") as f: # Open file in read mode
        return f.read()

### 3.4 Examine the results of the vector search
Given the text of the question, we can now perform a vector search and output the results in the notebook. An important detail here is the ability to interact with metadata (e.g. context source) which can be used to narrow down the search space and, more critically, for authorization. These approaches are out of scope of this lab. 

In [4]:
question = "What is ML Runtime?" ## (Swap with your own based on your dataset)

context_chunk, sources, score = get_nearest_chunk_from_pinecone_vectordb(index, question)
print("\nContext Chunk: ")
print(context_chunk)
print("\nContext Source(s): ")
print(sources)
print("\nPinecone Score: ")
print(score)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/cdsw/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
2024-05-02 03:28:40.908442: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 03:28:41.715220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 03:28:41.715253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 03:28:41.742961: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 


Context Chunk: 
ML RuntimesCloudera Docs ML Runtimes ML Runtimes are responsible for running data science workloads and intermediating       access to the underlying cluster.  CML allows you to run any code via an interactive session, scheduled job, or deployed model          or application. Data Scientists can use interactive sessions to explore data, or develop a          model. They can create jobs and schedule them to run at specified times or productionize          their work as a model to provide a REST endpoint or as an application that offers an          interactive data dashboard for business users. All of these workloads run inside an ML          Runtime container on top of Kubernetes.  Cloudera ML Runtimes are purpose built to serve a specific use-case. They are available          with a single editor (for example, Workbench, Jupyterlab), ship a single language Kernel          (for example, Python 3.8 or R 4.0), and have a set of UNIX tools and utilities or language        

### 3.5 Takeaways
* Vector search is a critical component of any LLM app using RAG architecture
* Cloudera's partner [Pinecone](https://www.pinecone.io/) provides a convenient SaaS offering for a Vector Database to support LLM RAG architecture
* Metadata from each entry in the Vector DB can be used to refine searches and add custom authorization frameworks

### Up Next: Go to Exercise 4
Where a gradio app is launched to complete the first iteration of the Q&A chat use case.